In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame({'key1':list('ababa'),
                  'key2': ['one','two','one','two','one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
print(df)

  key1 key2     data1     data2
0    a  one  1.129234  0.651835
1    b  two -1.421035  1.603570
2    a  one  2.290581 -0.756843
3    b  two -1.590281  1.766254
4    a  one  0.882883  0.042699


In [8]:
grouped=df['data1'].groupby(df['key1'])
list(grouped)

[('a',
  0    1.129234
  2    2.290581
  4    0.882883
  Name: data1, dtype: float64),
 ('b',
  1   -1.421035
  3   -1.590281
  Name: data1, dtype: float64)]

In [5]:
print(grouped.mean())

key1
a    1.434233
b   -1.505658
Name: data1, dtype: float64


In [12]:
states=np.array(['Ohio','California','California','Ohio','Ohio'])
years=np.array([2005,2005,2006,2005,2006])
#states第一层索引，years第二层分层索引

In [14]:
print(df['data1'].groupby([years,states]).mean())

2005  California   -1.421035
      Ohio         -0.230524
2006  California    2.290581
      Ohio          0.882883
Name: data1, dtype: float64


In [15]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.434233,-0.020770
b,-1.505658,1.684912


In [1]:
#输入必须是pandas
def mean_woe_target_encoder(train,test,target,col,n_splits=10):
    folds = StratifiedKFold(n_splits)

    y_oof = np.zeros(train.shape[0])
    y_oof_2= np.zeros(train.shape[0])
    y_test_oof = np.zeros(test.shape[0]).reshape(-1,1)
    y_test_oof2 = np.zeros(test.shape[0]).reshape(-1,1)

    splits = folds.split(train, target)
    
    for fold_n, (train_index, valid_index) in enumerate(splits):
        X_train, X_valid = train[col].iloc[train_index], train[col].iloc[valid_index]
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        clf=ce.target_encoder.TargetEncoder()
    
    #    dtrain = lgb.Dataset(X_train, label=y_train)
    #    dvalid = lgb.Dataset(X_valid, label=y_valid)
    
        #clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=1, early_stopping_rounds=500)
        clf.fit(X_train.values,y_train.values)    
        y_pred_valid = clf.transform(X_valid.values)
        y_oof[valid_index] = y_pred_valid.values.reshape(1,-1)

        tp=(clf.transform(test[col].values)/(n_splits*1.0)).values
        tp=tp.reshape(-1,1)
        y_test_oof+=tp    
    
        del X_train, X_valid, y_train, y_valid
        gc.collect()    
        
    for fold_n, (train_index, valid_index) in enumerate(splits):
        X_train, X_valid = train[col].iloc[train_index], train[col].iloc[valid_index]
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        clf=ce.woe.WOEEncoder()
    
    #    dtrain = lgb.Dataset(X_train, label=y_train)
    #    dvalid = lgb.Dataset(X_valid, label=y_valid)
    
        #clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=1, early_stopping_rounds=500)
        clf.fit(X_train.values,y_train.values)    
        y_pred_valid = clf.transform(X_valid.values)
        y_oof2[valid_index] = y_pred_valid.values.reshape(1,-1)
    
        tp=(clf.transform(test[col].values)/(n_splits*1.0)).values
        tp=tp.reshape(-1,1)
        y_test_oof2+=tp    
        del X_train, X_valid, y_train, y_valid
        gc.collect()     
    return y_oof,y_oof_2,y_test_oof,y_test_oof2

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
import pandas as pd

In [23]:
data_df = pd.DataFrame({'key1':list('ababa'),
                        'key2': ['one','two','one','two','one'],
                        'street_address': ['ac','ws','sd','qwed','wefQEW'],
                        'data1': np.random.randn(5),
                        'data2': np.random.randn(5)})

In [22]:
data_df

,key1,key2,street_address,data1,data2
0,a,0,ac,0.042363,-0.360161
1,b,1,ws,-1.314928,-0.478044
2,a,0,sd,0.542726,1.643066
3,b,1,qwed,-1.866147,-0.027763
4,a,0,wefQEW,0.393950,0.257604


In [21]:
le = LabelEncoder()
data_df['key2'] = le.fit_transform(data_df['key2'])

In [24]:
ohe = OneHotEncoder(categories='auto'
                    #,n_values='auto'
                    #, categorical_features='all'
                    , dtype=np.float64
                    , sparse=True
                    , handle_unknown='error'
                   )
ac = np.array(data_df['key2']).reshape(-1,1)
ac

array([['one'],
       ['two'],
       ['one'],
       ['two'],
       ['one']], dtype=object)

In [25]:
one_hot_matrix = ohe.fit_transform(ac)

In [26]:
for i in one_hot_matrix:
    print(i)
#    break

  (0, 0)	1.0
  (0, 1)	1.0
  (0, 0)	1.0
  (0, 1)	1.0
  (0, 0)	1.0


In [ ]:
#mean_encoder 方法
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from itertools import product

class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode

        :param n_splits: the number of splits used in mean encoding

        :param target_type: str, 'regression' or 'classification'

        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()

        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()

        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg({'mean': 'mean', 'beta': 'size'})
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)

        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new